In [1]:
! pip install py7zr pandas


   ---------------------------------------- 0.0/69.7 kB ? eta -:--:--
   ---------------------------------------- 69.7/69.7 kB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/369.1 kB ? eta -:--:--
   ------------------------------------ --- 337.9/369.1 kB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 369.1/369.1 kB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.2/1.8 MB 5.0 MB/s eta 0:00:01
   ----------- ---------------------------- 0.5/1.8 MB 6.6 MB/s eta 0:00:01
   ------------------ --------------------- 0.8/1.8 MB 6.6 MB/s eta 0:00:01
   ------------------------- -------------- 1.2/1.8 MB 6.7 MB/s eta 0:00:01
   ---------------------------------- ----- 1.5/1.8 MB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 6.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/46.6 kB ? eta -:--:--
   -----------------------

In [ ]:
# importation des packages
import requests
import pandas as pd
import numpy as np
import gzip
import py7zr
import io
import seaborn as sns
import os
import tempfile

## Admin express dataset: par territoire de la France Métropolitaine

In [ ]:

def extract_local_7z(filepath: str) -> str:
    if not os.path.isfile(filepath):
        raise FileNotFoundError(f"Fichier introuvable : {filepath}")

    tmp_dir = tempfile.mkdtemp()
    print(f"Extraction dans : {tmp_dir}")

    with py7zr.SevenZipFile(filepath, mode='r') as z:
        z.extractall(path=tmp_dir)

    print("Extraction terminée !")
    
    # Liste des fichiers
    print("\n Fichiers extraits :")
    for root, dirs, files in os.walk(tmp_dir):
        for f in files:
            print("   ", os.path.join(root, f))
    
    return tmp_dir



In [ ]:
# chemin vers le fichier local(à mettre dans github mais possible que ce soit trop lours pour y être téléchargé, pas encore trouvé d'API)
path_7z = "C:/Users/lisaw/Desktop/ENSAE/3A/Machine_learning/Project/ADMIN-EXPRESS_4-0__GPKG_LAMB93_FXX_2025-10-15.7z"

folder = extract_local_7z(path_7z)

In [ ]:
import geopandas as gpd
# Fichier principal
gpkg_path = "C:/Users/lisaw/AppData/Local/Temp/tmpynaoy8wa/ADMIN-EXPRESS_4-0__GPKG_LAMB93_FXX_2025-10-15/ADMIN-EXPRESS/1_DONNEES_LIVRAISON_2025-10-00142/ADE_4-0_GPKG_LAMB93_FXX-ED2025-10-15/ADE_4-0_GPKG_LAMB93_FXX-ED2025-10-15.gpkg"

gdf = gpd.read_file(gpkg_path)
gdf.head()

In [ ]:

len(gdf["code_insee_du_departement"].unique())
# 95 département, correspond avec les données du niveau d'argile

In [ ]:
# base des communes
gdf_communes = gpd.read_file(gpkg_path, layer="commune").to_crs(2154)
#base des départements
gdf_deps      = gpd.read_file(gpkg_path, layer="departement").to_crs(2154)
gdf_deps.head()

## Données BRGM

In [ ]:

# Décompresser le dossier 

def extract_7z(url: str):
    import requests
    from io import BytesIO

    # Téléchargement
    response = requests.get(url)
    response.raise_for_status()

    # Stockage temporaire
    data = BytesIO(response.content)

    # Extraction
    tmp_dir = tempfile.mkdtemp()
    with py7zr.SevenZipFile(data, mode='r') as z:
        z.extractall(path=tmp_dir)

    return tmp_dir

# Chercher chaque fichier .mbtiles dans le dossier
def find_mbtiles(folder):
    for root, _, files in os.walk(folder):
        for f in files:
            if f.endswith(".mbtiles"):
                return os.path.join(root, f)
    return None

In [ ]:
import sqlite3
import pandas as pd

# Lecture de chaque fichier .mbtiles 
def mbtiles_to_dfs(mbtiles_path):
    conn = sqlite3.connect(mbtiles_path)

    # Récupérer la liste des tables
    tables = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table';", conn
    )["name"].tolist()

    dfs = {}
    for t in tables:
        try:
            dfs[t] = pd.read_sql_query(f"SELECT * FROM {t}", conn)
        except Exception as e:
            print("Impossible de lire table", t, ":", e)

    conn.close()
    return dfs


In [ ]:
# ressort un dictionnaire de datatframes
def load_mbtiles_7z(url):
    folder = extract_7z(url)
    mbtiles = find_mbtiles(folder)
    if mbtiles is None:
        raise FileNotFoundError("Aucun fichier .mbtiles trouvé")
        
    print("Fichier mbtiles trouvé :", mbtiles)
    dfs = mbtiles_to_dfs(mbtiles)
    return dfs

url = "https://www.data.gouv.fr/api/1/datasets/r/c944be1e-06d6-46be-bf7d-9f9ad2b8ced9"

dfs = load_mbtiles_7z(url)

for name, df in dfs.items():
    print("----", name, "----")
    print(df.head())


In [ ]:
dfs["metadata"] # Métadonnées

In [ ]:
# Conversion du format pbf en geojson
! pip install mapbox-vector-tile mercantile geopandas shapely brotli

In [ ]:
import gzip
import mapbox_vector_tile

# Décompression du GZIP
def decode_mvt(tile_data: bytes) -> dict:
    """
    Décompresse GZIP puis décode MVT → dictionnaire Python
    """
    try:
        raw = gzip.decompress(tile_data)
        return mapbox_vector_tile.decode(raw)
    except Exception as e:
        raise RuntimeError(f"Erreur de décompression/décodage MVT: {e}")

# Conversion en GeoJSON (manipulation des données géométrique)
import geopandas as gpd
from shapely.geometry import shape

def mvt_dict_to_gdf(decoded: dict) -> gpd.GeoDataFrame:
    feats = []
    
    for layer_name, layer in decoded.items():
        for feat in layer["features"]:
            geom = shape(feat["geometry"])
            props = feat["properties"]
            props["__layer__"] = layer_name
            props["geometry"] = geom
            feats.append(props)

    return gpd.GeoDataFrame(feats, geometry="geometry", crs="EPSG:3857")

# conversion en geodataframe
def tiles_to_geodata(map_df, images_df, max_tiles=None):
    merged = map_df.merge(images_df, on=["zoom_level", "tile_id"])
    gdfs = []
    nbad = 0
    
    for i, row in merged.iterrows():
        if max_tiles and i >= max_tiles:
            break

        try:
            decoded = decode_mvt(row["tile_data"])
            gdf = mvt_dict_to_gdf(decoded)
            
            # Ajout champs z/x/y
            gdf["z"] = int(row["zoom_level"])
            gdf["x"] = int(row["tile_column"])
            gdf["y"] = int(row["tile_row"])
            
            gdfs.append(gdf)
        
        except Exception:
            nbad += 1
    
    if not gdfs:
        raise RuntimeError("Aucune tuile lisible !")
    
    print(f"Tuiles décodées : {len(gdfs)} | ignorées : {nbad}")
    return pd.concat(gdfs, ignore_index=True)

gdf_raw = tiles_to_geodata(dfs["map"], dfs["images"], max_tiles=50)
gdf_raw.head()



In [ ]:
# Affichage de la première tuile
#sample = dfs["images"].iloc[0]["tile_data"]
#decoded = decode_mvt(sample)
#decoded

In [ ]:
# Reprojection en WGS84
gdf_wgs84 = gdf_raw.to_crs(4326)
gdf_wgs84.head()


In [ ]:
print(gdf_wgs84.geom_type.value_counts())

## DAILY SWI

In [52]:
#Fonction pour récupérer les données d'un fichier csv.gz ( pour les données météorologiques )
def recuperer_donnees_gz(dataset_id, fichier_csv, sep):
    """
    Cette fonction télécharge les données d'un fichier CSV.GZ à partir d'une URL et les charge dans un DataFrame pandas.
    
    Paramètres:
    dataset_id (str): identifiant du fichier CSV.GZ.
    fichier_csv (str): Le nom du fichier CSV.GZ à enregistrer localement.
    sep ( str ) : le séparateur à utiliser
    
    Retourne:
    DataFrame: Un DataFrame pandas contenant les données du fichier CSV.
    """

    # URL de base pour accéder à l'API
    base_api = "https://www.data.gouv.fr/api/1/"

    # Chemin pour accéder aux enregistrements du dataset
    key_api = "datasets/r/"

    # Construction de l'URL complète
    url = f"{base_api}{key_api}{dataset_id}"


    response = requests.get(url)
    response.raise_for_status()
    with gzip.GzipFile(fileobj=io.BytesIO(response.content)) as gz:
        df = pd.read_csv(gz, sep = sep)
    
    return df

In [48]:
#liste_id=['eb0d6e42-cee6-4d7c-bc5b-646be4ced72e','33417617-c0dd-4513-804e-c3f563cb81b4','08ad5936-cb9e-4284-a6fc-36b29aca9607',
#          'ad584d65-7d2d-4ff1-bc63-4f93357ed196','10d2ce77-5c3b-44f8-bb46-4df27ed48595','da6cd598-498b-4e39-96ea-fae89a4a8a46',
#          '92065ec0-ea6f-4f5e-8827-4344179c0a7f','adcca99a-6db0-495a-869f-40c888174a57']
liste_id=['ad584d65-7d2d-4ff1-bc63-4f93357ed196','10d2ce77-5c3b-44f8-bb46-4df27ed48595','da6cd598-498b-4e39-96ea-fae89a4a8a46',
          '92065ec0-ea6f-4f5e-8827-4344179c0a7f','adcca99a-6db0-495a-869f-40c888174a57']
# année=[1960-1969,1970-1979,1980-1989,'1990-1999',2000-2009,2010-2019,2020-202510,20251001-20251107]

In [ ]:
# calcul des données mensuelles à partir des variables météorologiques
def compute_mean_swi(data_id, value_cols=None):
    df= recuperer_donnees_gz(data_id, '', ';')
    df["YEAR"] = df["DATE"].astype(str).str[:4].astype(int)
    df["MONTH"] = df["DATE"].astype(str).str[4:6]
    df=df.drop("DATE",axis=1)
    # Colonnes clés
    group_cols = ["LAMBX", "LAMBY", "YEAR", "MONTH"]

    # Déterminer les colonnes à moyenner
    if value_cols is None:
        # On prend toutes les colonnes numériques sauf YEAR/MONTH/LAMBX/LAMBY
        exclude = set(group_cols)
        num_cols = [c for c in df.select_dtypes(include="number").columns
                    if c not in exclude]
        value_cols = num_cols
     # Agrégation
    out = (
        df[group_cols + value_cols]
        .groupby(group_cols, as_index=False)
        .mean()
    )
     # Recréer une DATE mensuelle (au 1er jour du mois)
    
    return out

# extraction des données de 2020-2025
data20_25=compute_mean_swi(data_id='92065ec0-ea6f-4f5e-8827-4344179c0a7f')
data20_25.head()

,LAMBX,LAMBY,YEAR,MONTH,PRENEI,PRELIQ,T,FF,Q,DLI,...,RESR_NEIGE6,HTEURNEIGE,HTEURNEIGE6,HTEURNEIGEX,SNOW_FRAC,ECOULEMENT,WG_RACINE,WGI_RACINE,TINF_H,TSUP_H
0,600,24010,2020,01,0.0,3.370968,8.854839,6.019355,6.281355,2918.677419,...,0.0,0.0,0.0,0.0,0.0,0.0,0.314161,0.0,6.890323,10.603226
1,600,24010,2020,02,0.0,5.124138,9.568966,8.234483,6.346655,2912.062069,...,0.0,0.0,0.0,0.0,0.0,0.0,0.315621,0.0,7.627586,11.541379
2,600,24010,2020,03,0.0,2.164516,8.845161,6.816129,5.750774,2791.793548,...,0.0,0.0,0.0,0.0,0.0,0.0,0.313452,0.0,6.629032,11.303226
3,600,24010,2020,04,0.0,2.260000,12.263333,4.380000,7.068567,2902.943333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.283333,0.0,9.190000,15.906667
4,600,24010,2020,05,0.0,0.709677,14.567742,4.990323,7.591258,2905.525806,...,0.0,0.0,0.0,0.0,0.0,0.0,0.276097,0.0,11.245161,18.403226


In [208]:
#Fusion des tables
#liste_df=[]
#for id in liste_id:
#    data=compute_mean_swi(data_id=id)
#    liste_df.append(data)
#daily_swi_df=pd.concat(liste_df,ignore_index=True)
#daily_swi_df.shape

## SWI Uniforme

In [55]:
import os

# chemin vers le dossier à modifier, mais les données ont stockés dans github (open data donc ca ne dérange pas)
folder_path = "C:/Users/lisaw/Desktop/ENSAE/3A/Machine_learning/Project/MACHINE-LEARNING-FOR-CLIMATE-RISK/Data_swi_uniform"

dfs = []

for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)
        #print("Lecture :", file_path)
        df = pd.read_csv(file_path, sep=";")
        dfs.append(df)

swi_uniform_df = pd.concat(dfs, ignore_index=True)

print("Fusion terminée")
print(swi_uniform_df.shape)

#swi_uniform_df.to_csv("merged.csv", index=False)


Fusion terminée
(7005180, 5)


In [ ]:
#données de 1960 - 2024 sur toutes les mailles
swi_uniform_df["SWI_UNIF_MENS"] = (
    swi_uniform_df["SWI_UNIF_MENS"]
    .astype(str)                     # convertit en chaîne
    .str.replace(",", ".", regex=False)  # remplace les virgules par des points
    .astype(float)                   # convertit en float
)
swi_uniform_df.head()

,NUMERO,LAMBX,LAMBY,DATE,SWI_UNIF_MENS
0,2,641374,7106309,196001,0.863
1,2,641374,7106309,196002,0.876
2,2,641374,7106309,196003,0.856
3,2,641374,7106309,196004,0.757
4,2,641374,7106309,196005,0.673


### Jointure des points de la grille SAFRAN aux communes

In [71]:
# lien à modifier, les données sont stockées sur le repo
# utilisation de la longitude et de la latitude puis conversion en Lambert-93 (EPSG:2154) car les points de la grille utilisée n'ont pas les mêmes echelles que celles de ADMIN-EXPRESS
df_safran=pd.read_csv("C:/Users/lisaw/Desktop/ENSAE/3A/Machine_learning/Project/MACHINE-LEARNING-FOR-CLIMATE-RISK/coordonnees_grille_safran_lambert-2-etendu.csv", sep=";")
df_safran["LON_DG"] = df_safran["LON_DG"].astype(str).str.replace(",", ".", regex=False)
df_safran["LAT_DG"] = df_safran["LAT_DG"].astype(str).str.replace(",", ".", regex=False)
gdf_pts = gpd.GeoDataFrame(
    df_safran.copy(),
    geometry=gpd.points_from_xy(df_safran["LON_DG"], df_safran["LAT_DG"]),
    crs=4326
).to_crs(2154)
gdf_pts.head()

,LAMBX (hm),LAMBY (hm),LAT_DG,LON_DG,geometry
0,600,24010,48.3822,-4.96118,POINT (111539.179 6838790.165)
1,760,24170,48.5386,-4.76561,POINT (127654.757 6854651.667)
2,760,24090,48.467,-4.75585,POINT (127591.268 6846660.146)
3,760,24010,48.3953,-4.74612,POINT (127526.489 6838658.116)
4,760,23930,48.3237,-4.73641,POINT (127463.328 6830667.619)


In [72]:
# --- 3) Jointure spatiale point->commune
# nécessite shapely>=2, et une sindex (rtree/pygeos) pour la perf
pts_communes = gpd.sjoin(gdf_pts, gdf_communes[["code_insee","nom_officiel","geometry"]],
                         predicate="within", how="left")
pts_communes.head()

,LAMBX (hm),LAMBY (hm),LAT_DG,LON_DG,geometry,index_right,code_insee,nom_officiel
0,600,24010,48.3822,-4.96118,POINT (111539.179 6838790.165),NaN,NaN,NaN
1,760,24170,48.5386,-4.76561,POINT (127654.757 6854651.667),NaN,NaN,NaN
2,760,24090,48.467,-4.75585,POINT (127591.268 6846660.146),NaN,NaN,NaN
3,760,24010,48.3953,-4.74612,POINT (127526.489 6838658.116),10419.0,29201,Ploumoguer
4,760,23930,48.3237,-4.73641,POINT (127463.328 6830667.619),NaN,NaN,NaN


In [144]:
# Coordonnées à retirer car ils ne correspondents pas aux points terrestres de la France métropolitaine (pas de correspondance avec les communes en France)
pts_communes[pd.isna(pts_communes["code_insee"])][["LAMBX","LAMBY","geometry"]]

,LAMBX,LAMBY,geometry
0,600,24010,POINT (111539.179 6838790.165)
1,760,24170,POINT (127654.757 6854651.667)
2,760,24090,POINT (127591.268 6846660.146)
4,760,23930,POINT (127463.328 6830667.619)
9,840,23930,POINT (135456.69 6830602.157)
...,...,...,...
9882,11880,16810,POINT (1232344.463 6110018.501)
9885,11880,16570,POINT (1232133.752 6086042.284)
9886,11960,17450,POINT (1240897.106 6173866.185)
9887,11960,17290,POINT (1240756.254 6157889.64)


In [145]:
# Points à utilser pour la jointure avec les données du swi journalier
pts_communes_new=pts_communes.loc[pts_communes["code_insee"].notna(), ["LAMBX", "LAMBY","code_insee","nom_officiel","geometry"]]
pts_communes_new.head()

,LAMBX,LAMBY,code_insee,nom_officiel,geometry
3,760,24010,29201,Ploumoguer,POINT (127526.489 6838658.116)
5,760,23610,29168,Plogoff,POINT (127207.712 6798688.534)
6,840,24170,29178,Ploudalmézeau,POINT (135648.089 6854583.39)
7,840,24090,29119,Lanrivoaré,POINT (135584.389 6846592.83)
8,840,24010,29130,Locmaria-Plouzané,POINT (135520.121 6838591.69)


In [149]:
import pandas as pd
# ajouter les références de communes à chaque observation du swi journalier
def join_by_coordinates(table1, table2):
    """
    Jointure sur (LAMBX, LAMBY) entre table1 et table2 :
    - ajoute code_insee et nom_commune depuis table2
    - supprime les lignes de table1 sans correspondance
    
    table1 : DataFrame contenant LAMBX, LAMBY, + variables observées
    table2 : DataFrame contenant LAMBX, LAMBY, code_insee, nom_commune
    """
    merged = table1.merge(
        table2[["LAMBX", "LAMBY", "code_insee", "nom_officiel","geometry"]],
        on=["LAMBX", "LAMBY"],
        how="inner"   # garde uniquement les correspondances
    )
    return merged
daily_swi_communes=join_by_coordinates(data20_25, pts_communes_new)
daily_swi_communes.shape

(601510, 33)

In [150]:
daily_swi_communes["YEAR"].unique()
daily_swi_df_train= daily_swi_communes[
    daily_swi_communes["YEAR"].isin([2020, 2021, 2022, 2023, 2024])
]

In [ ]:
tables=[]
for id in liste_id:
    data=compute_mean_swi(id)
    data_communes=join_by_coordinates(data, pts_communes_new)
    tables.append(data_communes)

In [ ]:
### Jointure des données du swi uniforme aux communes

In [184]:
gdf_pts_swi = gpd.GeoDataFrame(
    swi_uniform_df,
    geometry=gpd.points_from_xy(swi_uniform_df["LAMBX"], swi_uniform_df["LAMBY"]),
    crs=2154
)
# --- 3) Jointure spatiale point->commune
# nécessite shapely>=2, et une sindex (rtree/pygeos) pour la perf
swi_communes = gpd.sjoin(gdf_pts_swi, gdf_communes[["code_insee","nom_officiel","geometry"]],
                         predicate="within", how="left")
swi_communes.head()

,NUMERO,LAMBX,LAMBY,DATE,SWI_UNIF_MENS,geometry,index_right,code_insee,nom_officiel
0,2,641374,7106309,196001,0.863,POINT (641374 7106309),NaN,NaN,NaN
1,2,641374,7106309,196002,0.876,POINT (641374 7106309),NaN,NaN,NaN
2,2,641374,7106309,196003,0.856,POINT (641374 7106309),NaN,NaN,NaN
3,2,641374,7106309,196004,0.757,POINT (641374 7106309),NaN,NaN,NaN
4,2,641374,7106309,196005,0.673,POINT (641374 7106309),NaN,NaN,NaN


In [185]:
#suppression des observations qui n'ont pas de correspondance avec les communes de la france métropolitaine
swi_communes_new=swi_communes.loc[swi_communes["code_insee"].notna(), ["LAMBX","LAMBY",	"DATE",	"SWI_UNIF_MENS","geometry","index_right","code_insee","nom_officiel"]]
swi_communes_new.head()

,LAMBX,LAMBY,DATE,SWI_UNIF_MENS,geometry,index_right,code_insee,nom_officiel
1560,657366,7106175,196001,0.869,POINT (657366 7106175),21923.0,59183,Dunkerque
1561,657366,7106175,196002,0.881,POINT (657366 7106175),21923.0,59183,Dunkerque
1562,657366,7106175,196003,0.859,POINT (657366 7106175),21923.0,59183,Dunkerque
1563,657366,7106175,196004,0.760,POINT (657366 7106175),21923.0,59183,Dunkerque
1564,657366,7106175,196005,0.677,POINT (657366 7106175),21923.0,59183,Dunkerque


In [186]:
swi_communes_new["YEAR"] = swi_communes_new["DATE"].astype(str).str[:4].astype(int)
swi_communes_new["MONTH"] = swi_communes_new["DATE"].astype(str).str[4:6]
swi_communes_new["YEAR"].unique()

array([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970,
       1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024])

In [187]:
swi_communes_new_train = swi_communes_new[
    swi_communes_new["YEAR"].isin([2020, 2021, 2022, 2023, 2024])
]


In [ ]:
# Jointure du swi et du swi uniforme

(6705660, 8)

In [201]:
daily_swi_df_train[(daily_swi_df_train["nom_officiel"]=="Plogoff") & (daily_swi_df_train["YEAR"]==2020)].head(20)

,LAMBX,LAMBY,YEAR,MONTH,PRENEI,PRELIQ,T,FF,Q,DLI,...,HTEURNEIGEX,SNOW_FRAC,ECOULEMENT,WG_RACINE,WGI_RACINE,TINF_H,TSUP_H,code_insee,nom_officiel,geometry
0,76000,2361000,2020,01,0.000000,2.764516,9.196774,5.532258,6.290258,2813.222581,...,0.0,0.0,0.000000,0.261290,0.0,7.419355,10.729032,29168,Plogoff,POINT (127207.712 6798688.534)
1,76000,2361000,2020,02,0.000000,3.700000,10.003448,8.265517,6.424241,2820.086207,...,0.0,0.0,0.000000,0.262207,0.0,8.337931,11.541379,29168,Plogoff,POINT (127207.712 6798688.534)
2,76000,2361000,2020,03,0.000000,1.890323,9.054839,5.545161,5.719323,2690.900000,...,0.0,0.0,0.000000,0.258710,0.0,6.651613,11.493548,29168,Plogoff,POINT (127207.712 6798688.534)
3,76000,2361000,2020,04,0.000000,1.720000,12.536667,3.526667,7.095300,2863.553333,...,0.0,0.0,0.000000,0.220467,0.0,8.876667,16.263333,29168,Plogoff,POINT (127207.712 6798688.534)
4,76000,2361000,2020,05,0.000000,1.009677,15.280645,3.583871,7.714677,2827.890323,...,0.0,0.0,0.000000,0.213806,0.0,11.925806,19.119355,29168,Plogoff,POINT (127207.712 6798688.534)
5,76000,2361000,2020,06,0.000000,3.056667,15.890000,3.983333,9.012933,3028.210000,...,0.0,0.0,0.000000,0.195767,0.0,13.730000,18.496667,29168,Plogoff,POINT (127207.712 6798688.534)
6,76000,2361000,2020,07,0.000000,0.777419,17.087097,3.935484,9.654323,3087.277419,...,0.0,0.0,0.000000,0.183387,0.0,14.216129,20.048387,29168,Plogoff,POINT (127207.712 6798688.534)
7,76000,2361000,2020,08,0.000000,2.658065,18.206452,4.041935,10.792710,3212.180645,...,0.0,0.0,0.000000,0.182903,0.0,15.745161,21.238710,29168,Plogoff,POINT (127207.712 6798688.534)
8,76000,2361000,2020,09,0.000000,1.440000,16.233333,3.343333,9.490033,3153.336667,...,0.0,0.0,0.000000,0.180567,0.0,13.250000,19.560000,29168,Plogoff,POINT (127207.712 6798688.534)
9,76000,2361000,2020,10,0.000000,4.293548,12.425806,5.929032,7.787258,3002.496774,...,0.0,0.0,0.000000,0.219645,0.0,10.216129,15.000000,29168,Plogoff,POINT (127207.712 6798688.534)


In [198]:
swi_communes_new_train[(swi_communes_new_train["YEAR"]==2020) & (swi_communes_new_train["nom_officiel"]=="Plogoff")].head(20)

,LAMBX,LAMBY,DATE,SWI_UNIF_MENS,geometry,index_right,code_insee,nom_officiel,YEAR,MONTH
1266660,127137,6798687,202001,1.014,POINT (127137 6798687),10388.0,29168,Plogoff,2020,01
1266661,127137,6798687,202002,1.026,POINT (127137 6798687),10388.0,29168,Plogoff,2020,02
1266662,127137,6798687,202003,0.998,POINT (127137 6798687),10388.0,29168,Plogoff,2020,03
1266663,127137,6798687,202004,0.731,POINT (127137 6798687),10388.0,29168,Plogoff,2020,04
1266664,127137,6798687,202005,0.643,POINT (127137 6798687),10388.0,29168,Plogoff,2020,05
1266665,127137,6798687,202006,0.424,POINT (127137 6798687),10388.0,29168,Plogoff,2020,06
1266666,127137,6798687,202007,0.292,POINT (127137 6798687),10388.0,29168,Plogoff,2020,07
1266667,127137,6798687,202008,0.249,POINT (127137 6798687),10388.0,29168,Plogoff,2020,08
1266668,127137,6798687,202009,0.204,POINT (127137 6798687),10388.0,29168,Plogoff,2020,09
1266669,127137,6798687,202010,0.494,POINT (127137 6798687),10388.0,29168,Plogoff,2020,10


In [202]:
import numpy as np

def distance_lambert93(x1, y1, x2, y2):
    """Distance en mètres entre deux points Lambert-93"""
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Exemple
d = distance_lambert93(127207.712, 6798688.534, 127137,6798687)
print(f"Distance = {d:.2f} m")


Distance = 70.73 m


In [189]:
#conversion en mètre
#daily_swi_df_train["LAMBX"]=daily_swi_df_train["LAMBX"]*100
#daily_swi_df_train["LAMBY"]=daily_swi_df_train["LAMBY"]*100


In [129]:
len(swi_communes_new_train['code_insee'].unique())

8057

In [ ]:
# Affectation du swi uniforme par plus proche voisin
pandas as pd
import numpy as np
from scipy.spatial import cKDTree

def join_swi_to_uniforme_ckdtree_from_geometry(swi_gdf, swi_uniforme_df, max_dist=8000):
    """
    Associe SWI à SWI_UNIFORME par commune, année et mois :
    - pour swi_gdf : coordonnées extraites de la colonne geometry (GeoDataFrame)
    - pour swi_uniforme_df : coordonnées dans LAMBX / LAMBY
    - si plusieurs points, appariement par plus proche voisin (KDTree)
    """
    results = []
    swi_gdf = gpd.GeoDataFrame(
    swi_gdf,
    geometry="geometry",
    crs="EPSG:2154")


    # Extraire coordonnées x,y à partir de la géométrie
    swi_gdf = swi_gdf.copy()
    swi_gdf["x"] = swi_gdf.geometry.x
    swi_gdf["y"] = swi_gdf.geometry.y

    # Boucle principale sur (YEAR, MONTH)
    for (year, month), g1 in swi_gdf.groupby(["YEAR", "MONTH"], sort=False):
        g2 = swi_uniforme_df.query("YEAR == @year and MONTH == @month")
        if g2.empty:
            continue

        g2_small = g2[["code_insee", "LAMBX", "LAMBY", "SWI_UNIF_MENS"]].copy()

        for code_insee, g1c in g1.groupby("code_insee", sort=False):
            g2c = g2_small[g2_small["code_insee"] == code_insee]
            if g2c.empty:
                continue

            # Cas 1 : un seul point dans chaque table
            if len(g1c) == 1 and len(g2c) == 1:
                g1c = g1c.assign(SWI_UNIF_MENS=g2c["SWI_UNIF_MENS"].values[0])
                results.append(g1c)
                continue

            # Cas 2 : plusieurs points → plus proche voisin via KDTree
            tree = cKDTree(g2c[["LAMBX", "LAMBY"]].values)
            dist, idx = tree.query(g1c[["x", "y"]].values, distance_upper_bound=max_dist)

            mask = np.isfinite(dist)
            swi_unif = np.full(len(g1c), np.nan)
            swi_unif[mask] = g2c["SWI_UNIF_MENS"].values[idx[mask]]

            g1c = g1c.assign(SWI_UNIF_MENS=swi_unif, dist_m=dist)
            results.append(g1c)

    merged = pd.concat(results, ignore_index=True)
    return merged

merged_df = join_swi_to_uniforme_ckdtree_from_geometry(
    swi_gdf=daily_swi_df_train,
    swi_uniforme_df=swi_communes_new_train,
    max_dist=12000  # 10 km
)

In [206]:
np.sum(merged_df["SWI_UNIF_MENS"].isna())

0

In [168]:
from scipy.spatial import cKDTree
import numpy as np

g1 = daily_swi_df_train[(daily_swi_df_train["nom_officiel"]=="Arles") & (daily_swi_df_train["YEAR"]==2020)].head(40)
g1 = gpd.GeoDataFrame(
    g1,
    geometry="geometry",
    crs="EPSG:2154"
)
g2 = swi_communes_new_train[(swi_communes_new_train["YEAR"]==2020) & (swi_communes_new_train["nom_officiel"]=="Arles")].head(40)
g1["x"]= g1.geometry.x
g1["y"]= g1.geometry.y
tree = cKDTree(g2[["LAMBX", "LAMBY"]])
dist, idx = tree.query(g1[["x", "y"]], distance_upper_bound=20000)
print(dist)


[  44.05118909   44.05118909   44.05118909   44.05118909   44.05118909
   44.05118909   44.05118909   44.05118909   44.05118909   44.05118909
   44.05118909   44.05118909 7995.43512455 7995.43512455 7995.43512455
 7995.43512455 7995.43512455 7995.43512455 7995.43512455 7995.43512455
 7995.43512455 7995.43512455 7995.43512455 7995.43512455   43.58846765
   43.58846765   43.58846765   43.58846765   43.58846765   43.58846765
   43.58846765   43.58846765   43.58846765   43.58846765   43.58846765
   43.58846765           inf           inf           inf           inf]
